In [ ]:
# example for text processing on CFPB data; ZU; April 2020

# watch for path: works for local machine path; replace it with actual. (code, data in different folders)

# summary of steps:
# loading libraries; data file into data frame
# remove rows that do not have narratives from the data frame
# perform some text preprocessing on the nrrative column
# put the processed text back into the shorter data frame
# save the data frame in csv file.


In [1]:
# importing libraries, etc
# reading CFPB csv data file; full, large, (until Oct 2019??)

import pandas as pd
from nltk.corpus import stopwords
import nltk
from nltk import trigrams
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('max_colwidth',2000)
path='C:\\Users\\Zsolt\\Box Sync\\Projects - Research\\CFPB\\Data\\'
cc=pd.read_csv(path+'Consumer_Complaints.csv')

In [2]:
# size, type:

print(cc.shape)
print(type(cc))

(1416910, 18)
<class 'pandas.core.frame.DataFrame'>


In [3]:
# date conversion

cc['Date received']=pd.to_datetime(cc['Date received'])

In [4]:
print(type(cc['Date received']))

<class 'pandas.core.series.Series'>


In [62]:
# create empty rows list (rows with no narratives in them); view numbers

cc['Consumer complaint narrative'].isnull().value_counts()
cc['Consumer complaint narrative']=cc['Consumer complaint narrative'].fillna('missing')
ccEmptyList=cc[(cc['Consumer complaint narrative']=='missing')].index.tolist()
ccEmptyList

[0,
 1,
 2,
 3,
 5,
 6,
 7,
 11,
 13,
 14,
 16,
 17,
 18,
 20,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 30,
 31,
 33,
 34,
 36,
 37,
 38,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 50,
 51,
 52,
 54,
 57,
 58,
 59,
 60,
 61,
 62,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 75,
 76,
 77,
 79,
 81,
 83,
 85,
 88,
 89,
 90,
 91,
 92,
 94,
 96,
 97,
 101,
 102,
 105,
 106,
 107,
 108,
 109,
 111,
 112,
 115,
 116,
 117,
 119,
 120,
 121,
 124,
 125,
 126,
 128,
 130,
 132,
 135,
 137,
 138,
 139,
 140,
 141,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 153,
 154,
 156,
 158,
 159,
 160,
 164,
 165,
 171,
 172,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 186,
 187,
 188,
 190,
 191,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 204,
 205,
 206,
 209,
 210,
 211,
 214,
 217,
 219,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 235,
 236,
 238,
 239,
 241,
 242,
 243,
 244,
 247,
 248,
 249,
 250,
 251,
 252,
 254,
 255,
 256

In [9]:
# drop empty rows
cc_narrative=cc.drop(ccEmptyList)
# reset index
cc_narrative=cc_narrative.reset_index(drop=True)
# show dataframe size
cc_narrative.shape

(454661, 18)

In [11]:
# convert narrative column to lower case, show top rows
cc_narrative['Consumer complaint narrative']=cc_narrative['Consumer complaint narrative'].str.replace("[^a-zA-Z]", " ").str.lower()
cc_narrative['Consumer complaint narrative'].head()

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    i am dissatisfied with the current outcome of a dispute that was initiated with discover card regarding a single transaction that o

In [12]:
# add some words to stop_words list from nltk; may need more.
# print stop_words data type, list

original_stop_words=set(stopwords.words('english'))
added_stop_words={'xxxx','xxxx-xxxx','xxxx xxxx xxxx','i','xx'}
stop_words=added_stop_words.union(original_stop_words)
print(type(stop_words))
stop_words

<class 'set'>


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [13]:
# remove stop_words, view head

cc_narrative['Consumer complaint narrative'] =cc_narrative['Consumer complaint narrative'].apply(lambda x: ' '.join([w for w in x.split() if w not in stop_words])) 
cc_narrative['Consumer complaint narrative'].head()

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       dissatisfied current outcome dispute initiated discover card regarding single transaction occurred amount corresponded discover card least four times since enclosed attachment complaint believe credit card issuer violated consumer protection laws failing implement special rule credit card purchase protection despite overwhelming paperwork evidence submitted shows merchant conducted business bad faith less favorable consumer sustained monetary loss result merch

In [15]:
# change to list type, view

cc_tokenized_text_narrative=cc_narrative['Consumer complaint narrative'] .apply(lambda x:x.split())
cc_tokenized_text_narrative.head()

0    [dissatisfied, current, outcome, dispute, initiated, discover, card, regarding, single, transaction, occurred, amount, corresponded, discover, card, least, four, times, since, enclosed, attachment, complaint, believe, credit, card, issuer, violated, consumer, protection, laws, failing, implement, special, rule, credit, card, purchase, protection, despite, overwhelming, paperwork, evidence, submitted, shows, merchant, conducted, business, bad, faith, less, favorable, consumer, sustained, monetary, loss, result, merchants, bad, faith, intent, patiently, utilized, internal, discover, card, dispute, process, past, three, months, credit, card, issuer, always, favoring, merchant, repeatedly, submitted, irrefutable, paperwork, evidence, shown, merchant, conducted, business, bad, faith, tried, good, faith, address, complaint, merchant, discover, card, believe, receive, favorable, outcome]
1                                                                                                    

In [16]:
# load stemmer, stem, view head (takes a while; several minutes; 10+? )

from nltk.stem.porter import*
stemmer=PorterStemmer()
cc_tokenized_text_narrative=cc_tokenized_text_narrative.apply(lambda x:[stemmer.stem(i) for i in x])
cc_tokenized_text_narrative.head()

0    [dissatisfi, current, outcom, disput, initi, discov, card, regard, singl, transact, occur, amount, correspond, discov, card, least, four, time, sinc, enclos, attach, complaint, believ, credit, card, issuer, violat, consum, protect, law, fail, implement, special, rule, credit, card, purchas, protect, despit, overwhelm, paperwork, evid, submit, show, merchant, conduct, busi, bad, faith, less, favor, consum, sustain, monetari, loss, result, merchant, bad, faith, intent, patient, util, intern, discov, card, disput, process, past, three, month, credit, card, issuer, alway, favor, merchant, repeatedli, submit, irrefut, paperwork, evid, shown, merchant, conduct, busi, bad, faith, tri, good, faith, address, complaint, merchant, discov, card, believ, receiv, favor, outcom]
1                                                                                                                                                                                                                           

In [17]:
# make narratives strings again
# replace the narrative column in the cc_narrative dataframe with the processed strings (lower case, stop words removed, stemmed)
# view column head

for i in range(len(cc_tokenized_text_narrative)):
    cc_tokenized_text_narrative[i]=' '.join(cc_tokenized_text_narrative[i])
cc_narrative['Consumer complaint narrative']=cc_tokenized_text_narrative
cc_narrative['Consumer complaint narrative'].head()

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    dissatisfi current outcom disput initi discov card regard singl transact occur amount correspond discov card least four time sinc enclos attach complaint believ credit card issuer violat consum protect law fail implement special rule credit card purchas protect despit overwhelm paperwork evid submit show merchant conduct busi bad faith less favor consum sustain monetari loss result merchant bad faith intent patient util intern discov card disput process past three month credit card issuer alway favor merchant 

In [18]:
# view cc_narrative data frame head
cc_narrative.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2016-03-04,Credit card,NaN,Billing disputes,NaN,dissatisfi current outcom disput initi discov card regard singl transact occur amount correspond discov card least four time sinc enclos attach complaint believ credit card issuer violat consum protect law fail implement special rule credit card purchas protect despit overwhelm paperwork evid submit show merchant conduct busi bad faith less favor consum sustain monetari loss result merchant bad faith intent patient util intern discov card disput process past three month credit card issuer alway favor merchant repeatedli submit irrefut paperwork evid shown merchant conduct busi bad faith tri good faith address complaint merchant discov card believ receiv favor outcom,NaN,DISCOVER BANK,NV,891XX,NaN,Consent provided,Web,03/04/2016,Closed with explanation,Yes,Yes,1816726
1,2018-03-03,"Credit reporting, credit repair services, or other personal consumer reports",Credit reporting,Incorrect information on your report,Information belongs to someone else,item show credit report previous remov differ collect agenc base burden proof debt even exist would appear debt sold practic highli illeg demand remov report immedi,Company has responded to the consumer and the CFPB and chooses not to provide a public response,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",VA,245XX,NaN,Consent provided,Web,03/03/2018,Closed with explanation,Yes,NaN,2831821
2,2019-01-02,Debt collection,I do not know,Attempts to collect debt not owed,Debt was result of identity theft,saw credit report collect know sinc incur debt first collect compani said close account return account client mean debt,Company believes complaint caused principally by actions of third party outside the control or direction of the company,"Cascade Capital, LLC",FL,334XX,Older American,Consent provided,Web,01/18/2019,Closed with explanation,Yes,NaN,3114357
3,2018-12-23,"Credit reporting, credit repair services, or other personal consumer reports",Credit reporting,Improper use of your report,Credit inquiries on your report that you don't recognize,capit one auto finac unknown inquir credit report,NaN,CAPITAL ONE FINANCIAL CORPORATION,MD,212XX,NaN,Consent provided,Web,12/23/2018,Closed with explanation,Yes,NaN,3108212
4,2018-04-13,Credit card or prepaid card,General-purpose credit card or charge card,Problem with a purchase shown on your statement,Credit card company isn't resolving a dispute about a purchase on your statement,small truck compani one truck need repair hire mechan repair told us instal new batteri new starter truck mechan state work guarante paid td bank credit card repair driver went start truck work next day truck work start tri call text mechan let know truck work even though previou day said fix truck text phone record show reach mechan respons hire anoth mechan fix truck found part put truck first mechan wrong file disput transact td bank credit card even though paid first mechan disput amount amount wrong part credit card compani request proof record second estim work done truck overnight packag credit card compani within day two disput depart request second estim disput depart state submit second estim document fax second estim date disput depart receiv overnight packag call followup tuesday disput depart state never receiv second estim need disput even though fax confirm transmiss state fax sent sent fax onxx deadlin document receiv fax confirm sheet state fax sent td bank credit card state never receiv second estim time close disput even though everyth request time want failur end receiv fax even though proof compani lost protect logo,NaN,TD BANK US HOLDING COMPANY,NJ,NaN,NaN,Consent provided,Web,04/16/2018,Closed with monetary relief,Yes,NaN,2874325


In [77]:
# write cc_narrative data frame to csv file

cc_narrative.to_csv(path+'complaints-narrativeonly-stemmed.csv',index=False)

In [100]:
# importing libraries, etc
# reading CFPB above processed csv data file

import pandas as pd
pd.set_option('max_colwidth',2000)
path='C:\\Users\\Zsolt\\Box Sync\\Projects - Research\\CFPB\\Data\\'
df=pd.read_csv(path+'complaints-narrativeonly-stemmed.csv')
df['Date received']=pd.to_datetime(df['Date received'])
df.shape

(454661, 18)

In [101]:
# df2 dataframe: Navient rows only; size
df2 = df[(df['Company']=='Navient Solutions, LLC.')]
df2=df2.reset_index(drop=True)
df2.shape

(13744, 18)

In [102]:
list2 = list(df2['Consumer complaint narrative'])
len(list2)


13744

In [103]:
seen = {}
dupes = []
i = 0
DuplList = []

for x in list2:
    if x not in seen:
        seen[x] = 1
    else:
        if seen[x] == 1:
            dupes.append(x)
        seen[x] += 1
        DuplList.append(i)
    i = i+1
len(seen), len(dupes), len(DuplList), i

(13722, 14, 22, 13744)

In [104]:
DuplList

[827,
 2063,
 4141,
 4300,
 5459,
 6597,
 6649,
 6790,
 7180,
 9388,
 9709,
 10319,
 10829,
 11191,
 11327,
 11477,
 12066,
 12204,
 12506,
 13086,
 13372,
 13545]

In [105]:
# drop empty rows
df3=df2.drop(DuplList)
# reset index
df3=df3.reset_index(drop=True)
# show dataframe size
df3.shape

(13722, 18)

In [107]:
df3.to_csv(path+'Navient-no-duplicates.csv',index=False)